In [184]:
import numpy as np
import torch
class BloatWareConvTranspose2D():
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, dtype=None):
        
        if isinstance(in_channels, int) and in_channels > 0:
            self.in_channels = in_channels
        else:
            raise ValueError("Invalid in_channels")
        
        if isinstance(out_channels, int) and out_channels > 0:
            self.out_channels = out_channels
        else:
            raise ValueError("Invalid out_channels")
        
        if isinstance(groups, int) and groups > 0:
            self.groups = groups
        else:
            raise ValueError("Invalid groups")

        if isinstance(stride, int) and stride > 0:
            self.stride = stride
        else:
            raise ValueError("Invalid stride")
        
        if isinstance(padding, int) and padding > -1:
            self.padding = padding
        else:
            raise ValueError("Invalid padding")
            
        if isinstance(output_padding, int) and output_padding > -1:
            self.output_padding = output_padding
        else:
            raise ValueError("Invalid output_padding")
            
        if isinstance(dilation, int) and dilation > 0:
            self.dilation = dilation
        else:
            raise ValueError("Invalid dilation")
            
        if not((self.in_channels % self.groups == 0) and (self.out_channels % self.groups == 0)):
            raise ValueError("in_channels and out_channels must both be divisible by groups")
        
        if (self.output_padding >= self.dilation and self.output_padding >= self.stride) or (self.output_padding >= self.dilation and self.output_padding >= self.stride):
            raise ValueError("output_padding should be smaller than dilation or stride")

        if bias == True:
          self.bias = torch.rand(self.out_channels)
        else:
          self.bias = torch.zeros(self.out_channels)
    
        if isinstance(kernel_size, int):
            self.weight = torch.rand(
                self.in_channels,
                self.out_channels,
                kernel_size,
                kernel_size,
            )
            self.kernel_size = kernel_size
        else:
            raise ValueError("kernel size must be int or tuple")

        self.dtype = dtype
    
    def forward(self, input_tensor):
        result = []

        for l in range(self.out_channels):
    
          feature_map = torch.zeros((input_tensor.shape[1]-1)*self.stride + self.dilation * (self.kernel_size-1)+1, (input_tensor.shape[2]-1)*self.stride  + self.dilation * (self.kernel_size-1)+1 ) #генерация пустой feature-map
          for c in range (self.in_channels):
    
            for i in range (0, input_tensor.shape[1]):  #проход по всем пикселям изображения
              for j in range (0, input_tensor.shape[2]):
    
                val = input_tensor[c][i][j]
                proizv = val*self.weight[c][l]
    
                zero_tensor = torch.zeros((self.weight.shape[2]-1)*self.dilation+1, (self.weight.shape[3]-1)*self.dilation+1)
    
                for a in range (0, zero_tensor.shape[0], self.dilation):
                  for b in range (0, zero_tensor.shape[1], self.dilation):
                    zero_tensor[a][b] = proizv[a//self.dilation][b//self.dilation]
    
                res = np.add((zero_tensor), feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1])
                feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1] = res
    
    
          result.append(np.add(feature_map, np.full((feature_map.shape), self.bias[l])))
    
    
        for t in range(len(result)):
          if self.output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, self.output_padding, 0, self.output_padding), 0)
            result[t] = pad_func(result[t])
    
          result[t] = result[t][0+self.padding:result[t].shape[0]-self.padding, 0+self.padding:result[t].shape[1]-self.padding]

        return np.array(result)

In [185]:
torchConvTranspose2D = torch.nn.ConvTranspose2d(15, 5, 3, stride=1, padding=1, dilation=1)
input_image = torch.randn(15, 50, 50)
output = torchConvTranspose2D(input_image)

In [186]:
myConvTranspose2D = BloatWareConvTranspose2D(15, 5, 3, stride=1, padding=1, dilation=1)

In [187]:
myConvTranspose2D.weight = torchConvTranspose2D.weight.detach().numpy()
myConvTranspose2D.bias = torchConvTranspose2D.bias.detach().numpy()

In [188]:
torchConvTranspose2D.weight.shape[0]

15

In [189]:
output_mock = myConvTranspose2D.forward(input_image)

In [190]:
output_mock

array([[[-0.32785314,  0.6804811 , -0.7160098 , ..., -0.03277095,
         -2.111997  , -1.8800467 ],
        [ 0.03005262, -1.025222  , -1.5796629 , ...,  1.5506848 ,
         -0.5474681 , -0.189187  ],
        [-0.4405887 , -0.72080725, -0.33276588, ...,  0.63289374,
          1.134695  ,  1.291157  ],
        ...,
        [-1.1405551 ,  0.6169815 ,  0.0402675 , ...,  1.5148016 ,
         -1.1224909 ,  0.33574766],
        [ 0.26439023,  0.25257677,  0.6103852 , ...,  1.3353134 ,
          0.30463696, -0.30290973],
        [-0.08855709,  0.9013244 ,  0.5237863 , ..., -0.4800341 ,
         -1.054776  , -0.5607661 ]],

       [[-0.47302523, -0.70303595,  0.7516928 , ...,  1.4544461 ,
          0.78200454,  0.75267035],
        [-0.39958665,  0.15369278,  0.28326315, ...,  0.24692625,
         -0.16545637, -0.32509685],
        [-0.5051445 ,  2.0281272 , -0.05625352, ..., -0.6630189 ,
         -0.90240747,  1.6585252 ],
        ...,
        [ 0.43847096,  0.25700855,  1.7181423 , ...,  

In [191]:
output

tensor([[[-0.3279,  0.6805, -0.7160,  ..., -0.0328, -2.1120, -1.8800],
         [ 0.0301, -1.0252, -1.5797,  ...,  1.5507, -0.5475, -0.1892],
         [-0.4406, -0.7208, -0.3328,  ...,  0.6329,  1.1347,  1.2912],
         ...,
         [-1.1406,  0.6170,  0.0403,  ...,  1.5148, -1.1225,  0.3357],
         [ 0.2644,  0.2526,  0.6104,  ...,  1.3353,  0.3046, -0.3029],
         [-0.0886,  0.9013,  0.5238,  ..., -0.4800, -1.0548, -0.5608]],

        [[-0.4730, -0.7030,  0.7517,  ...,  1.4544,  0.7820,  0.7527],
         [-0.3996,  0.1537,  0.2833,  ...,  0.2469, -0.1655, -0.3251],
         [-0.5051,  2.0281, -0.0563,  ..., -0.6630, -0.9024,  1.6585],
         ...,
         [ 0.4385,  0.2570,  1.7181,  ...,  1.2603, -0.3124,  0.1266],
         [-0.5397, -0.5068, -1.6265,  ...,  1.5853,  0.8843,  0.1346],
         [-0.7900,  1.1839, -0.2239,  ...,  0.9059, -0.5105, -0.3782]],

        [[-0.7668,  0.0188,  0.6478,  ...,  0.2258,  0.6017,  0.6251],
         [-1.3819, -0.2811, -0.5021,  ...,  1

In [192]:
result_test = output.detach().numpy().astype("float16") == output_mock.astype(
    "float16"
)

In [193]:
result_test

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

In [196]:
def myTranspConv2d(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #всегда 1 шаг
    stride = 1

    #добавление отступов и padding в входной матрице
    pad = kernel_size - 1
    result_matrix = []
    for matr in matrix:
      zero_tensor = np.zeros((((matr.shape[0]-1)*(transp_stride)+1), ((matr.shape[1]-1)*(transp_stride)+1)))
      for a in range (0, zero_tensor.shape[0], transp_stride):
        for b in range (0, zero_tensor.shape[1], transp_stride):
          zero_tensor[a][b] = matr[a//(transp_stride)][b//(transp_stride)]

      pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
      result_matrix.append(pad_matr)
    matrix = torch.tensor(result_matrix)

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

    #инвертирование ядра для ConvTranspose2d
    filter_for_transpose = []
    for j in range(out_channels):
      filter_in = []
      for i in range(in_channels):
        filter_in.append(np.flip(np.array(filter[j][i])))
      filter_for_transpose.append(filter_in)

    filter_for_transpose = torch.tensor(filter_for_transpose)
    filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)



    spisok = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels):
            val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias
      spisok.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))


    return np.array(spisok), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_val))

  return Svertca

In [197]:
import numpy as np
import torch
import sys
import os

tensor1 = torch.rand(3, 5, 6)

myFunction = myTranspConv2d(in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, bias=True,)
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))


In [198]:
myResult

'[[[0.83 0.99 1.52 1.6  2.25 1.57 2.36 1.54 2.18 1.11 1.65 1.41 1.58]\n  [0.88 1.05 1.59 1.67 2.15 1.73 2.29 1.76 1.95 1.19 1.67 1.5  1.43]\n  [1.14 1.33 2.7  1.74 3.51 2.11 3.96 1.87 3.58 2.08 3.73 2.01 2.51]\n  [0.96 1.36 1.85 1.22 2.13 1.89 2.33 1.5  2.29 2.03 2.77 1.82 1.9 ]\n  [2.52 2.3  3.81 1.31 3.87 2.85 4.31 2.01 4.49 2.42 5.05 2.33 3.25]\n  [1.85 2.02 2.48 1.34 2.66 2.5  3.19 1.72 2.76 2.13 2.94 2.17 2.08]\n  [2.79 2.61 4.23 1.83 4.99 2.66 5.19 2.1  4.33 2.03 3.78 1.97 2.02]\n  [1.86 2.07 2.89 1.74 2.79 2.15 2.91 1.99 2.48 1.64 2.02 1.24 1.29]\n  [1.99 1.59 3.18 2.07 3.78 1.84 4.47 2.76 4.95 1.85 4.05 2.42 2.72]\n  [1.01 1.08 1.88 1.72 2.12 1.3  2.48 2.57 3.05 1.65 2.87 2.26 2.36]\n  [1.07 0.82 1.65 1.11 1.62 0.93 2.67 1.38 2.93 0.97 2.95 1.23 1.53]]]'

In [199]:
torchResult

'[[[0.83 0.99 1.52 1.6  2.25 1.57 2.36 1.54 2.18 1.11 1.65 1.41 1.58]\n  [0.88 1.05 1.59 1.67 2.15 1.73 2.29 1.76 1.95 1.19 1.67 1.5  1.43]\n  [1.14 1.33 2.7  1.74 3.51 2.11 3.96 1.87 3.58 2.08 3.73 2.01 2.51]\n  [0.96 1.36 1.85 1.22 2.13 1.89 2.33 1.5  2.29 2.03 2.77 1.82 1.9 ]\n  [2.52 2.3  3.81 1.31 3.87 2.85 4.31 2.01 4.49 2.42 5.05 2.33 3.25]\n  [1.85 2.02 2.48 1.34 2.66 2.5  3.19 1.72 2.76 2.13 2.94 2.17 2.08]\n  [2.79 2.61 4.23 1.83 4.99 2.66 5.19 2.1  4.33 2.03 3.78 1.97 2.02]\n  [1.86 2.07 2.89 1.74 2.79 2.15 2.91 1.99 2.48 1.64 2.02 1.24 1.29]\n  [1.99 1.59 3.18 2.07 3.78 1.84 4.47 2.76 4.95 1.85 4.05 2.42 2.72]\n  [1.01 1.08 1.88 1.72 2.12 1.3  2.48 2.57 3.05 1.65 2.87 2.26 2.36]\n  [1.07 0.82 1.65 1.11 1.62 0.93 2.67 1.38 2.93 0.97 2.95 1.23 1.53]]]'